In [7]:
from bigml.api import BigML
from pandas import read_csv
df = read_csv('https://oml-data.s3.amazonaws.com/kaggle-give-me-credit-train.csv')
df = df.fillna(0) 
df['MonthlyPerPerson'] = df['MonthlyIncome'] / ( df['NumberOfDependents'] + 1 )
df.loc[df.age > 80, 'isOld'] = '1' 
df.loc[df.age <= 80, 'isOld'] = '0'
df['Monthlydebt'] = df['DebtRatio'] *  df['MonthlyIncome']
df['LatePayment'] = df['NumberOfTime30-59DaysPastDueNotWorse'] + df['NumberOfTimes90DaysLate'] + df['NumberOfTime60-89DaysPastDueNotWorse']
df['NumberOfCredits'] = df['NumberRealEstateLoansOrLines'] + df['NumberOfOpenCreditLinesAndLoans']

df.to_csv('GMSC.csv')
# df[:20]

#Connexion BigML avec seule mention du projet après modification du docker-compose
#environment: 


# Source
api = BigML(project='project/5d94a3e85a213962e2000304')
source = api.create_source('GMSC.csv')
api.ok(source)

# Dataset
origin_dataset = api.create_dataset(source)
api.ok(origin_dataset)

# Split
train_dataset = api.create_dataset(
    origin_dataset, {"name": "GMSC-Training",
                     "sample_rate": 0.8, "seed": "my seed"})
test_dataset = api.create_dataset(
    origin_dataset, {"name": "GMSC-Test",
                     "sample_rate": 0.8, "seed": "my seed",
                     "out_of_bag": True})

#Modèle choisit : ensemble
ensemble = api.create_ensemble(train_dataset , {"objective_field" : "SeriousDlqin2yrs"})

evaluation = api.create_evaluation(ensemble, test_dataset)

#Batch prediction 
batch_prediction = api.create_batch_prediction(ensemble, test_dataset,{"all_fields": True,
                                                                        "header": True,
                                                                      "confidence": True,
                                                                      "probabilities":True})
api.ok(batch_prediction)

#fichier csv batch prédiction
api.download_batch_prediction(batch_prediction, filename='BatchPrediction/GMSC_Prediction_Ensemble.csv')

'BatchPrediction/GMSC_Prediction_Ensemble.csv'

In [8]:
#Exporter modèle sur Kaggle

df = read_csv('BatchPrediction/GMSC_Prediction_Ensemble.csv')

,field1,Unnamed: 0,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,...,SeriousDlqin2yrs,MonthlyPerPerson,isOld,Monthlydebt,LatePayment,NumberOfCredits,SeriousDlqin2yrs.1,confidence,0 probability,1 probability
0,4,5,0.907239,49,1,0.024926,63588,7,0,1,...,0,63588.0,0,1584.975094,1,8,0,0.80763,0.84224,0.15776
1,19,20,0.602794,25,0,0.065868,333,2,0,0,...,0,333.0,0,21.934132,0,2,0,0.93340,0.95304,0.04696
2,26,27,0.052436,58,0,0.097672,8333,22,0,1,...,0,8333.0,0,813.902326,0,23,0,0.97771,0.98507,0.01493
3,27,28,0.034421,69,0,0.042383,2500,17,0,0,...,0,1250.0,0,105.957617,0,17,0,0.98572,0.99045,0.00955
4,30,31,0.704074,28,1,0.155201,4200,8,0,0,...,0,4200.0,0,651.844801,1,8,0,0.85964,0.88539,0.11461


In [31]:
df2 = read_csv('https://oml-data.s3.amazonaws.com/kaggle-give-me-credit-test.csv')
df2 = df2.fillna(0) 
df2['MonthlyPerPerson'] = df2['MonthlyIncome'] / ( df2['NumberOfDependents'] + 1 )
df2.loc[df2.age > 80, 'isOld'] = '1' 
df2.loc[df2.age <= 80, 'isOld'] = '0'
df2['Monthlydebt'] = df2['DebtRatio'] *  df2['MonthlyIncome']
df2['LatePayment'] = df2['NumberOfTime30-59DaysPastDueNotWorse'] + df2['NumberOfTimes90DaysLate'] + df2['NumberOfTime60-89DaysPastDueNotWorse']
df2['NumberOfCredits'] = df2['NumberRealEstateLoansOrLines'] + df2['NumberOfOpenCreditLinesAndLoans']

df2.to_csv('KaggleGMSC.csv')

,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents,MonthlyPerPerson,isOld,Monthlydebt,LatePayment,NumberOfCredits
0,1,0.0,0.885519,43,0,0.177513,5700.0,4,0,0,0,0.0,5700.000000,0,1011.822487,0,4
1,2,0.0,0.463295,57,0,0.527237,9141.0,15,0,4,0,2.0,3047.000000,0,4819.472759,0,19
2,3,0.0,0.043275,59,0,0.687648,5083.0,12,0,1,0,2.0,1694.333333,0,3495.312354,0,13
3,4,0.0,0.280308,38,1,0.925961,3200.0,7,0,2,0,0.0,3200.000000,0,2963.074038,1,9
4,5,0.0,1.000000,27,0,0.019917,3865.0,4,0,0,0,1.0,1932.500000,0,76.980082,0,4
5,6,0.0,0.509791,63,0,0.342429,4140.0,4,0,0,0,1.0,2070.000000,0,1417.657571,0,4
6,7,0.0,0.587778,50,0,1048.000000,0.0,5,0,0,0,3.0,0.000000,0,0.000000,0,5
7,8,0.0,0.046149,79,1,0.369170,3301.0,8,0,1,0,1.0,1650.500000,0,1218.630830,1,9
8,9,0.0,0.013527,68,0,2024.000000,0.0,4,0,1,0,0.0,0.000000,0,0.000000,0,5
9,10,0.0,1.000000,23,98,0.000000,0.0,0,98,0,98,0.0,0.000000,0,0.000000,294,0


In [32]:
# Source
api = BigML(project='project/5d94a3e85a213962e2000304')
source = api.create_source('KaggleGMSC.csv')
api.ok(source)

True

In [34]:

# Dataset
kaggle_dataset = api.create_dataset(source)
api.ok(kaggle_dataset)

{'code': 200,
 'resource': 'dataset/5db1ac3952996320280009d4',
 'location': 'https://bigml.io/andromeda/dataset/5db1ac3952996320280009d4',
 'object': {'all_fields': True,
  'category': 0,
  'cluster': None,
  'cluster_status': False,
  'code': 200,
  'columns': 18,
  'configuration': None,
  'configuration_status': False,
  'correlations': {},
  'created': '2019-10-24T13:50:49.069000',
  'creator': 'clementgombeaud',
  'credits': 8.886545181274414,
  'dataset_origin_status': True,
  'description': '',
  'download': {'code': 0,
   'excluded_input_fields': [],
   'header': True,
   'input_fields': [],
   'message': '',
   'preview': [],
   'separator': ','},
  'evaluation': None,
  'excluded_fields': [],
  'field_types': {'categorical': 1,
   'datetime': 0,
   'items': 0,
   'numeric': 17,
   'preferred': 17,
   'text': 0,
   'total': 18},
  'fields': {'000000': {'column_number': 0,
    'datatype': 'int32',
    'name': 'field1',
    'optype': 'numeric',
    'order': 0,
    'preferred': T

In [35]:
kaggle_test = api.create_batch_prediction(ensemble, kaggle_dataset,{"all_fields": True,
                                                                    "header": True,
                                                                    "confidence": True, 
                                                                    "probabilities":True})
api.ok(kaggle_test)

True

In [37]:
api.download_batch_prediction(kaggle_test, filename='BatchPrediction/EnvoiKaggle.csv')

'BatchPrediction/EnvoiKaggle.csv'

In [38]:
prediction = read_csv('BatchPrediction/EnvoiKaggle.csv', index_col=False)

,field1,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,...,NumberOfDependents,MonthlyPerPerson,isOld,Monthlydebt,LatePayment,NumberOfCredits,SeriousDlqin2yrs.1,confidence,0 probability,1 probability
0,0,1,0,0.885519,43,0,0.177513,5700,4,0,...,0,5700.000000,0,1011.822487,0,4,0,0.92853,0.95157,0.04843
1,1,2,0,0.463295,57,0,0.527237,9141,15,0,...,2,3047.000000,0,4819.472759,0,19,0,0.93228,0.95203,0.04797
2,2,3,0,0.043275,59,0,0.687648,5083,12,0,...,2,1694.333333,0,3495.312354,0,13,0,0.97527,0.98378,0.01622
3,3,4,0,0.280308,38,1,0.925961,3200,7,0,...,0,3200.000000,0,2963.074038,1,9,0,0.92085,0.94183,0.05817
4,4,5,0,1.000000,27,0,0.019917,3865,4,0,...,1,1932.500000,0,76.980082,0,4,0,0.88051,0.90595,0.09405
5,5,6,0,0.509791,63,0,0.342429,4140,4,0,...,1,2070.000000,0,1417.657571,0,4,0,0.94825,0.96118,0.03882
6,6,7,0,0.587778,50,0,1048.000000,0,5,0,...,3,0.000000,0,0.000000,0,5,0,0.93465,0.95140,0.04860
7,7,8,0,0.046149,79,1,0.369170,3301,8,0,...,1,1650.500000,0,1218.630830,1,9,0,0.94578,0.96338,0.03662
8,8,9,0,0.013527,68,0,2024.000000,0,4,0,...,0,0.000000,0,0.000000,0,5,0,0.99527,0.99705,0.00295
9,9,10,0,1.000000,23,98,0.000000,0,0,98,...,0,0.000000,0,0.000000,294,0,0,0.46696,0.54328,0.45672


In [40]:
from pandas import DataFrame
kaggle_prediction=DataFrame()
kaggle_prediction['Id']=prediction['Unnamed: 0']
kaggle_prediction['Probability']=prediction['1 probability']

,Id,Probability
0,1,0.04843
1,2,0.04797
2,3,0.01622
3,4,0.05817
4,5,0.09405
5,6,0.03882
6,7,0.04860
7,8,0.03662
8,9,0.00295
9,10,0.45672


In [41]:
import kaggle
kaggle_prediction_file="kaggle_prediction.csv"
kaggle_prediction.to_csv(kaggle_prediction_file, index=False)
kaggle.api.competition_submit(kaggle_prediction_file, "BigML ensemble", "GiveMeSomeCredit")

100%|██████████| 1.65M/1.65M [00:02<00:00, 600kB/s] 


Successfully submitted to Give Me Some Credit